# DATA VISUALIZATION: INTERACTIVE MAPS

#### Import required libraries.

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
from folium import plugins

In [15]:
from math import sin, cos, sqrt, atan2, radians

#### Doanload the data file. 

In [4]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL, header = 0)
spacex_df.columns

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long'],
      dtype='object')

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
# Create a subset without the outcome. 
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df.head()

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


#### Create the map of the launch sites in relation to the NASA Johnson Space Center. 

In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Mark the NASA Johnson Space center with a pop-up and and an icon. 
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# Add the launch sites to the starter map. 
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values with a popup label.
for i in range(len(launch_sites_df)):
    coordinate = [launch_sites_df.iloc[i,1], launch_sites_df.iloc[i,2]]
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(launch_sites_df.iloc[i,0]))
    marker = folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df.iloc[i,0], ))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

#### Add a cluster of markers for each outcome. 

In [10]:
marker_cluster = MarkerCluster()

In [12]:
# Add a column for marker color, conditional on the class, green = success, red = failure. 
spacex_df["marker_color"] = ["green" if x == 1 else "red" for x in spacex_df["class"].values]
spacex_df[["Launch Site"]].value_counts()

Launch Site 
CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: count, dtype: int64

In [13]:
# Add marker_cluster to the current site_map
site_map.add_child(marker_cluster)

# Add a cluster marker for each launch, green = success, red = failed, at each launch site. 
for index, record in spacex_df.iterrows():
    coordinate = [spacex_df.iloc[index,1], spacex_df.iloc[index,2]]
    marker = folium.Marker(coordinate, popup='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % spacex_df.iloc[index,0], icon=folium.Icon(color='white',icon_color=spacex_df.iloc[index,4])).add_to(marker_cluster)

site_map

#### Show distances to landmarks. 

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map.
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [16]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
# Calculate the distance to the nearest coastline, 28.56294, -80.56788.
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_site_lat, launch_site_lon = 28.56322, -80.57683
coastline_lat, coastline_lon = 28.56294, -80.56788
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.8748996024953078

In [18]:
# Add a marker of the nearest coastline point.
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)
site_map

In [19]:
# Draw a line between the site and the point. 
lines=folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)